In [1]:
%run ../../common/import_all.py

from common.setup_notebook import set_css_style, setup_matplotlib, config_ipython
config_ipython()
setup_matplotlib()
set_css_style()

# Convolutional Neural Networks

Convolutional neural networks (aka *CNNs*) are [deep networks](deep.ipynb) such that the network is not fully connected: not every neuron is connected to every other one. These types of networks are particularly suited for working on *images* data as you can map regions of the image to specific neurons. Their architecture is indeed specifically designed to deal with image input data, and in fact neurons are arranged to follow the geometrical distributions of images data: there are the width and height dimensions, plus the one for the colour channel. 

The reason to use convolutional networks (whose specifics follow here) to deal with images is that "regular" fully connected nets would not replicate the geometry of the input data and would, more importantly, need too many parameters to be trained with success in a typical case. Also, we know that artificial neural networks have been inspired by biology; CNNs in particular have been conceived out of an inspiration from the visual cortex, where different neurons respond to different regions of the input space (see receptive fields below) and that visual neurons are organised in a matrix format, also see the experiment by Hubel and Wiesel in the '50s and '60s [[4](#huber-wiesel) where they have demonstrated this. 

CNNs learn to recognise subsequent hierarchical levels of shapes in the image, eventually managing to distinguish between, say, [the image of a dog and that of a cat](https://medium.com/@curiousily/tensorflow-for-hackers-part-iii-convolutional-neural-networks-c077618e590b).

The name *convolution* is somehow borrowed from the main [mathematical operation](https://en.wikipedia.org/wiki/Convolution) they rely upon, which is based on the multiplication between parameters (weights/bias) and input vectors in the activation function. CNNs' architectures are built as sequences of convolutional layer followed by pooling layer (see below), plus a fully connected layer at the end. There are variations as to what can go in between and so to how many convolutional layers are stacked and how they are alternated with other types or layers. The final fully connected layer is the one responsible for the final result.

## Snippets of history

For a bit of history, have a read at the seminal paper by LeCun et. al [[3]](#lecun) and watch [this video](https://www.youtube.com/watch?v=FwFduRA_L6Q) about the first CNN (*LeNet*, by LeCun) trained to recognise handwritten digits back n 1993, it's quite funny. This network has been utilised by the USA postal system to automatically read ZIP codes, in the 1990s!

Also, [this page](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html) collects a chronology of all the networks built to classify images, on various standard dataset, such as MNIST of CIFAR. The reality is, since deep learning became a thing at scale starting from the middle of the 2000's, we are assisting to a breakthrough in the history of science and technology in general, because these days CNNs are allowing for the realisation of tasks making for a new "summer of AI", which many people more qualified than be believe is this time here to stay. 

## Features of CNNs

### What's the convolution

> TODO

### Local Receptive Fields

In a typical convolutional structure, input neurons take the pixels of the image and hidden neurons are such that each of them only communicates to a portion of the input neurons, namely a region of the input image. Regions of the image do overlap in the sense that different hidden neurons see some of the same input neurons as the regions shift. This is the concept of *local receptive fields*, brilliantly illustrated in chapter 6 of Nielsen's book [[1]](#nielsen), where the following images are taken. Specifically, these images illustrate the concept via the example of a 28X28 input image (it's the classic example of the handwritten digits recognition from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)) and a 24X24 first hidden layer where each neuron deals with a region 5X5 (*filter*, or *receptive field*). Note that the output of a layer with the filter is called *feature map*. 

<figure style="float:center;">
  <img src="../../imgs/lrf1.png" width="500" align="center" style="margin:20px 50px"/>
  <figcaption>Image from the book by M Nielsen [[1]](#nielsen). The first neuron in the hidden neuron deals with a local region of the input image.</figcaption>
</figure>

<figure style="float:center;">
  <img src="../../imgs/lrf2.png" width="500" align="center" style="margin:0px 50px"/>
  <figcaption>Image from the book by M Nielsen [[1]](#nielsen). The second neuron in the hidden layer deals with a shifted (by one neuron) region of the input image.</figcaption>
</figure>

This means that for an image of shape $n \times n$, and using a kernel $m \times m$, with $m < n$, via this operation you end up with an output which is $(n - m + 1) \times (n - m + 1)$. 

We have illustrate the concept here with the assumption that the kernel will shift by one pixel every time. That's the typical situation, but there can be situations where you want the kernel to shift by a higher number of pixels, a quantity that is called the *stride*. The other parameter is the *padding*. We've seen that the application of a convolutional layer reduces the dimensionality of the data; if this is best to be avoided you can set some padding of chosen size, that is, add some 0 pixels around the input matrix borders so that effectively you're passing a bigger image. See these excellent blogs [[6]](#deshpande) on this.

### Shared weights (and biases)

In a CNN, the parameters (weights/biases) are shared by all neurons in the same hidden layer and this implies that all these neurons recognise (learn) the same feature, for example, vertical bars, at their respective location of interest. This makes for the translational invariance of CNNs and drastically reduces the potential number of parameters they'd have to learn otherwise. 


Because detecting one feature is not enough, multiple feature maps are assembled at the same level, so that each convolutional layer of neurons can detect multiple different features. Again, feature maps are the result of the application of the localisation into receptive fields and correspond to having the same shared parameters. See the following figure. 

<figure style="float:left;">
  <img src="../../imgs/cnn_featmaps.png" width="500" align="left" style="margin:20px 50px"/>
  <figcaption>Image from the book by M Nielsen [[1]](#nielsen). 20 feature maps the network has learned, each devoted to a feature. Each map is a matrix 5X5 of weights: the darker the colour, the heavier the weight there, which implies a stronger response of the network on the input image region at that location. The maps clearly shows the network has learned graphical patterns, for example in the third one on the top row, there is a vertical bar top right.
 
   What features a CNN actually learn isn't a trivial thing to ascertain, as in, the networks learns to recognise patterns on its own, but it's non trivial to imagine which ones beforehand. That's their power. 
 </figcaption>
</figure>

### Pooling layers

CNNs are completed by the presence of pooling layers beyond the convolutional ones, right after those. Pooling layers have the function of reducing the information/dimensionality the preceding convolutional layer outputs. In a max-pooling layer with 2X2 receptive field for instance (a usual suspect), the output of the convolutional layer is shrunk into a smaller matrix where each 2X2 region of pixels is reduced to one pixel with their maximum. 

Pooling is applied to each of the output feature maps. This procedure greatly reduces dimensionality and then the number of parameters, keeping only the relevant information and discarding precision. 

Max-pooling is a very common type of pooling but there are other ones, like average-pooling, where the average gets spit instead of the maximum, or $L_2$-pooling, where the square root of the sum of squares is spit. 

Through a pooling layer, an input $n \times n$ with a pooling kernel $2 \times 2$ will lead to an output $n/2 \times n/2$.

> TODO typical structure: sequence of conv/maxpooling, then dense ...
> TODO describe famous networks

### Channels

https://www.quora.com/What-do-channels-refer-to-in-a-convolutional-neural-network

### Dilated convolution

http://www.inference.vc/dilated-convolutions-and-kronecker-factorisation/

> TODO separate types of layers instead of types of networks and explain better why you put pooling layers after convolutional ones elsewhere

## References

1. <a name="nielsen"></a> M Nielsen, [**Neural Networks and Deep Learning**](http://neuralnetworksanddeeplearning.com/chap6.html), chapter 6
2. <a name="stanford"></a> [**Convolutional Neural Netoworks for visual recognition**](http://cs231n.github.io/convolutional-networks/), Stanford CS class by A Karpathy and F F Li
3. <a name="lecun"></a> Y LeCun, L Bottou, Y Bengio, P Haffner, [**Gradient-based learning applied to document recognition**](http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf), *Proc. of the IEEE*, 1998
4. [**A beginner guide to understanding neural networks**](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/)
5. [**A page on the experiment by Huber and Wiesel**](https://knowingneurons.com/2014/10/29/hubel-and-wiesel-the-neural-basis-of-visual-perception/)
6. <a name="deshpande"></a> **A beginner's guide to understanding convolutional neural networks**, [part 1](https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/) and [part 2](https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/), blogs by A Deshpande